In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from numba import jit
import math
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from fbprophet import Prophet
import pickle as pkl

train_file = '../input/hacklive-4/Train.csv'
test_file = '../input/hacklive-4/Test.csv'


df = pd.read_csv(train_file, low_memory=False, parse_dates=['Date'], index_col='ID')
tdf = pd.read_csv(test_file, low_memory=False, parse_dates=['Date'], index_col='ID')

df = pd.concat([df, tdf])
# tdf['Open'] = tdf['Close'] = tdf['High'] = tdf['Low'] = np.float64()
stock_dfs = []
for stock in tqdm(tdf.stock.unique(), total=len(tdf.stock.unique())):
# for stock in tqdm([1], total=1):
    
    forecast_dfs = []
    for val in ['Open', 'High', 'Low', 'Close']:
        train_df = pd.DataFrame({'ds': df.loc[df['stock'] == stock]['Date'].values , 'y': df.loc[df['stock'] == stock][val].values})

        holidays = pd.DataFrame({'holiday': 'holiday', 'ds': np.append(df.loc[(df['stock'] == stock) & (df['holiday'] == 1 )]['Date'].values, tdf.loc[(tdf['stock'] == stock) & (tdf['holiday'] == 1 )]['Date'].values)})


        m = Prophet(yearly_seasonality=True, holidays=holidays)
        m.fit(train_df)

        future = m.make_future_dataframe(periods=41 + 3 + 17, freq='D', include_history=True)
        forecast = m.predict(future)

        
        
        l = ['yhat', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper']
        l_final = [val + '_hat', val + '_trend', val + '_lower', val + '_upper', val + '_trend_lower', val + '_ trend_upper']
        forecast[val] = forecast.yhat
        for col in range(len(l)):
            forecast[l_final[col]] = forecast[l[col]]
        forecast = forecast[forecast['ds'].isin(df.Date.values)]
#         print(forecast.columns)
        forecast.drop(forecast.columns.difference(l_final), 1, inplace=True)
        
        
        
        forecast['ID'] = df[df['stock'] == stock].index

        forecast_dfs.append(forecast)
    stock_dfs.append(forecast_dfs[0].merge(forecast_dfs[1], on='ID').merge(forecast_dfs[2], on='ID').merge(forecast_dfs[3], on='ID'))

    


  8%|▊         | 8/103 [01:48<21:07, 13.34s/it]

In [ ]:
pd.concat(stock_dfs).shape, df.shape

In [ ]:
df = pd.concat(stock_dfs).set_index('ID', drop=True).join(df)

In [ ]:
df = df.drop(columns=['Open', 'High', 'Low'])

In [ ]:
df

In [ ]:
# df = pd.concat([df, tdf])

In [ ]:
df.to_csv('super_train.csv')

In [ ]:
from IPython.display import FileLink
FileLink('super_train.csv')

In [ ]:
! cat super_train.csv